# OpenAI Badminton Analysis - Refactored Version

This notebook demonstrates the refactored badminton analysis system using the new modular architecture.
The code has been updated to use the Phase 2 refactored components.

In [ ]:
#!pip install openai

from openai import OpenAI
import sys
import os
import csv

# Add the badminton package to the path
sys.path.insert(0, '/Users/chanakyd/work/vdark/badminton')

# Import legacy prompt modules (these still work with the refactored system)
import badminton.llm_analysis.bd_prompt as bd_prompt
import badminton.llm_analysis.shot_classification_prompt as scp

# Import the NEW refactored VideoPoseDataset (recommended)
from badminton.data.video_pose_dataset import VideoPoseDataset

# Alternative: You can still use the old import (with deprecation warning)
# from badminton.utilities.visualization_utilities import VideoPoseDataset

# Import utility functions
from badminton.utilities.coco_keypoints import create_keypoints_dict

# Import new modular components (optional - for advanced usage)
from badminton.data.pose_data_loader import PoseDataLoader
from badminton.visualization.pose_visualizer import PoseVisualizer
from badminton.features.pose_feature_extractor import PoseFeatureExtractor
from badminton.analysis.shot_descriptor import ShotDescriptor

## Setup Video and Pose Data

The refactored system maintains the same interface, so existing code works without changes.

In [ ]:
# Define file paths
video_file = "VB_DATA/poses/05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.mp4"
pose_file = "VB_DATA/poses/05_Drop_Shot/2022-09-01_17-49-36_dataset_set1_058_003682_003721_B_05.csv"

# Create VideoPoseDataset using the refactored version
# This now uses the modular architecture internally but maintains the same interface
vpd = VideoPoseDataset(poses_path=pose_file, video_path=video_file)

print(f"Dataset loaded successfully!")
print(f"Number of frames: {len(vpd)}")
print(f"Video metadata: {vpd.get_dataset_summary()}")

## Video Annotation

The `annotate_video_with_poses` method works exactly the same as before.

In [ ]:
# Create annotated video - same interface as before
vpd.annotate_video_with_poses(
    output_path="annotated_video_refactored.mp4", 
    include_bboxes=False, 
    players=['green', 'blue']
)

print("Annotated video created: annotated_video_refactored.mp4")

## Data Access

All existing data access patterns continue to work. The refactored system provides the same interface.

In [ ]:
# Access player data - same as before
print("Player A data (first frame):")
print(vpd.playera[0])  # Still works!

print("\nPlayer B data (first frame):")
print(vpd.playerb[0])  # Still works!

# New method: Get specific frame data
frame_data_a, frame_data_b = vpd.data_loader.get_frame_data(0)
print("\nUsing new modular approach:")
print(f"Frame 0 - Player A bbox: {frame_data_a[0]}")
print(f"Frame 0 - Player B bbox: {frame_data_b[0]}")

## Feature Extraction

Poselet extraction works the same as before, but now uses the modular feature extractor internally.

In [ ]:
# Extract poselets - same interface as before
poselets_green = vpd.get_poselets_for_player(player='green')
poselets_blue = vpd.get_poselets_for_player(player='blue')

print(f"Extracted {len(poselets_green)} frames of poselets for green player")
print(f"First frame poselets: {poselets_green[0]}")

# New: Use the modular feature extractor directly
feature_extractor = PoseFeatureExtractor()
player_data = vpd.data_loader.get_player_data('green')
poselets_direct = feature_extractor.extract_poselets_for_player(player_data)

print(f"\nDirect extraction: {len(poselets_direct)} frames")
print(f"Results match: {poselets_green == poselets_direct}")

## Shot Description Generation

Shot description generation maintains the same interface but uses the new modular shot descriptor.

In [ ]:
# Generate shot description - same interface as before
shot_description = vpd.get_shot_description_for_player(player='green')

print("Shot description for green player:")
print(shot_description[:500] + "..." if len(shot_description) > 500 else shot_description)

# New: Use the modular shot descriptor directly
shot_descriptor = ShotDescriptor()
poselets = vpd.get_poselets_for_player('green')
description_direct = shot_descriptor.generate_shot_description(poselets, poses_path=pose_file)

print(f"\nDescriptions match: {shot_description == description_direct}")

## New Analysis Features

The refactored system provides additional analysis capabilities.

In [ ]:
# New: Analyze shot patterns
shot_analysis = vpd.analyze_shot_pattern(player='green')
print("Shot pattern analysis:")
print(f"Frame count: {shot_analysis['frame_count']}")
print(f"Most common poselets: {shot_analysis['poselet_summary']['most_common_poselets']}")

# New: Get feature summary
feature_summary = vpd.feature_extractor.get_poselet_summary(poselets_green)
print(f"\nFeature summary: {feature_summary}")

## Using Individual Modules

The refactored system allows you to use individual components for more flexibility.

In [ ]:
# Use individual modules for custom workflows

# 1. Load data only
data_loader = PoseDataLoader(pose_file)
print(f"Loaded {len(data_loader)} frames of pose data")

# 2. Extract features only
extractor = PoseFeatureExtractor()
green_data = data_loader.get_player_data('green')
features = extractor.extract_poselets_for_player(green_data)
print(f"Extracted features for {len(features)} frames")

# 3. Analyze shots only
analyzer = ShotDescriptor()
analysis = analyzer.analyze_shot_pattern(features)
print(f"Analysis complete: {analysis['frame_count']} frames analyzed")

# 4. Create visualizations only (if you have video frames)
visualizer = PoseVisualizer()
print("Visualizer ready for custom rendering")

## OpenAI Integration

The OpenAI integration works exactly the same as before.

In [ ]:
# OpenAI client setup (add your API key)
client = OpenAI(
    api_key="your-api-key-here"  # Replace with your actual API key
)

# Generate prompt using the same method as before
prompt = scp.SC_BASE_PROMPT + scp.SC_INPUT_PROMPT + vpd.get_shot_description_for_player(player='green')

print("Prompt generated successfully!")
print(f"Prompt length: {len(prompt)} characters")
print("\nFirst 200 characters of prompt:")
print(prompt[:200] + "...")

In [ ]:
# Make OpenAI API call (uncomment and add your API key to use)
# response = client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "user", "content": prompt}
#     ],
#     max_tokens=500
# )
# 
# print("OpenAI Response:")
# print(response.choices[0].message.content)

print("OpenAI integration ready - add your API key to test!")

## Migration Notes

### What Changed:
- **Internal Architecture**: The `VideoPoseDataset` now uses modular components internally
- **New Import**: Recommended to use `from badminton.data.video_pose_dataset import VideoPoseDataset`
- **Additional Features**: New analysis methods and direct access to individual modules

### What Stayed the Same:
- **All existing methods** work exactly as before
- **Same interface** for `get_shot_description_for_player()`, `get_poselets_for_player()`, etc.
- **Same data access** via `vpd.playera`, `vpd.playerb`, etc.
- **Same OpenAI integration** patterns

### Benefits of Refactoring:
- **Better maintainability**: Each component has a single responsibility
- **Enhanced testability**: Individual modules can be tested separately
- **Improved extensibility**: Easy to add new features or modify existing ones
- **Code reusability**: Modules can be used independently
- **Better documentation**: Each module is well-documented with clear interfaces
